<a href="https://colab.research.google.com/github/K-107/BERTsDay/blob/main/BertTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU 정보 보기
! nvidia-smi --query | fgrep 'Product Name'

    Product Name                          : Tesla P100-PCIE-16GB


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 리포지토리 복제
! git clone https://github.com/K-107/BERTsDay.git

fatal: destination path 'BERTsDay' already exists and is not an empty directory.


In [2]:
# 밑의 모듈은 텐서플로 1.x로 작성되었으므로 버전 변경
! pip install --upgrade tensorflow==1.15.0rc3

Requirement already up-to-date: tensorflow==1.15.0rc3 in /usr/local/lib/python3.7/dist-packages (1.15.0rc3)


## pretrained BERT 모델을 모듈로 export

ETRI에서 사전훈련한 BERT의 체크포인트를 가지고 BERT 모듈을 만드는 과정.

In [49]:
! python /content/BERTsDay/Bert_fine_tuning/export_korbert/bert_to_module.py -i "/content/drive/MyDrive/3_Slot_Tagging_Proj/004_bert_eojeol_tensorflow" -o "/content/drive/MyDrive/3_Slot_Tagging_Proj"

## 데이터 준비

모델을 훈련하기 위해 필요한 seq.in, seq.out이라는 2가지 파일을 만드는 과정.



In [6]:
%cd /content/drive/MyDrive/3_Slot_Tagging_Proj/
#!python make_data.py train_data.txt > train_data.txt

/content/drive/MyDrive/3_Slot_Tagging_Proj


In [7]:
#1. 텍스트 파일에서 랜덤한 문장만 추출하는 명령어
!shuf -n 1280000 ./output_sample2.txt > ./sample/temp_0.txt

In [8]:
# 2. 공백 문자 여러개를 하나로 줄이는 명령어
! sed 's/  */ /g' ./sample/temp_0.txt > ./sample/temp_1.txt

In [9]:
#3. 앞, 뒤의 공백 문자를 없애는 명렁어
#앞 :
! sed 's/^ //g' ./sample/temp_1.txt > ./sample/temp_2.txt
#뒤 :
! sed 's/ $//g' ./sample/temp_2.txt > ./sample/temp_3.txt

In [10]:
#4. 중복 제거하는 명령어 (두 가지 방법 모두 동일합니다.)
! sort -u ./sample/sample_3.txt > ./sample/sample_1M.txt

# 임시 파일들 제거
! rm ./sample/temp_0.txt
! rm ./sample/temp_1.txt
! rm ./sample/temp_2.txt
! rm ./sample/temp_3.txt

tcmalloc: large alloc 1706696704 bytes == 0x55bf1d954000 @  0x7fb2873a41e7 0x55bf1b40f718 0x55bf1b40e5a1 0x7fb286d82bf7 0x55bf1b40f02a


In [11]:
# 실행 전에 소스 수정 필요함
! python /content/BERTsDay/Bert_fine_tuning/prepare_data.py -i ./sample/sample_1M.txt -o ./sample/

### Fine-tuing 훈련

어떻게 하면 train_bert_finetuning.py 코드를 실행할 수 있는지 코드 내부의 parser을 참조하여 작성하세요.
  
그리고 소스 파일 내에서 bert 모델의 경로를 고치세요

In [ ]:
# 실행 전에 소스 수정 필요함
! python /content/BERTsDay/Bert_fine_tuning/train_bert_finetuning.py -t "./sample/" -s ./fine_tuned -e 10000 -bs 128 -tp bert


2021-04-12 07:43:20.840381: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-12 07:43:20.845775: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-12 07:43:20.845981: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564c35a60680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-12 07:43:20.846011: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-12 07:43:20.847842: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-12 07:43:20.959477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-12 07:43:20.960311:

## 모델 평가

TODO - 위의 내용처럼 어떻게 하면 eval_bert_finetuned.py 코드를 실행할 수 있는지 코드 내부의 parser을 참조하여 작성하세요.
테스트의 결과는 --model에 넣어준 모델 경로 아래의 test_results에 저장된다.

## Inference (임의의 문장을 모델에 넣어보기)

TODO - eval_bert_finetuned.py를 참고하여 한 문장씩 넣어서 모델이 내뱉는 결과물을 볼 수 있도록 inference.py 코드를 완성하세요.  
python inference.py --model {훈련된 모델이 저장된 경로}  
예시: python inference.py --model saved_model/  
모델 자체가 용량이 커서 불러오는 데까지 시간이 걸림  
"Enter your sentence:"라는 문구가 나오면 모델에 넣어보고 싶은 문장을 넣어 주면 됨  
quit라는 입력을 넣어 주면 종료  